# Coleta de dados do A* Manhattan

In [ ]:
import os
import numpy as np
import csv
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import random
import time
import tracemalloc
from collections import deque
import pandas as pd
import sys
import heapq

def heuristica_manhattan(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def a_estrela_manhattan(labirinto, inicio, fim):
    altura, largura = len(labirinto), len(labirinto[0])
    aberta = []
    heapq.heappush(aberta, (0, inicio))
    veio_de = {}
    custo_ate_agora = {inicio: 0}
    nos_visitados = set()

    tracemalloc.start()
    tempo_inicio = time.perf_counter()

    while aberta:
        _, atual = heapq.heappop(aberta)
        nos_visitados.add(atual)

        if atual == fim:
            break

        for dx, dy in [(-1,0), (1,0), (0,-1), (0,1)]:
            nx, ny = atual[0] + dx, atual[1] + dy
            vizinho = (nx, ny)
            if 0 <= nx < largura and 0 <= ny < altura and labirinto[ny][nx] == 1:
                novo_custo = custo_ate_agora[atual] + 1
                if vizinho not in custo_ate_agora or novo_custo < custo_ate_agora[vizinho]:
                    custo_ate_agora[vizinho] = novo_custo
                    prioridade = novo_custo + heuristica_manhattan(fim, vizinho)
                    heapq.heappush(aberta, (prioridade, vizinho))
                    veio_de[vizinho] = atual

    tempo_fim = time.perf_counter()
    memoria_usada = tracemalloc.get_traced_memory()[1]
    tracemalloc.stop()

    caminho = []
    atual = fim
    while atual != inicio:
        caminho.append(atual)
        atual = veio_de.get(atual)
        if atual is None:
            caminho = []
            break
    if caminho:
        caminho.append(inicio)
        caminho.reverse()

    return {
        "caminho": caminho,
        "nos_visitados": list(nos_visitados),
        "tempo_ms": (tempo_fim - tempo_inicio) * 1000,
        "memoria_bytes": memoria_usada,
        "comprimento": len(caminho),
        "qtd_nos_visitados": len(nos_visitados)
    }

## Resolução de lote de labirintos

In [ ]:
def executar_astar_em_lote(caminho_labirintos, tamanho_nome, output_csv):
    resultados = []

    for i in range(1, 21):
        lab_path = os.path.join(caminho_labirintos, f"labirinto_{i:02d}.npy")
        if not os.path.exists(lab_path):
            print(f"Labirinto não encontrado: {lab_path}")
            continue

        labirinto = np.load(lab_path)
        entrada = (0, 1)
        saida = (labirinto.shape[1] - 1, labirinto.shape[0] - 2)

        resultado = a_estrela_manhattan(labirinto, entrada, saida)
        print(f"Labirinto {i}/20")
        resultados.append([
            "astar_Manhattan",
            tamanho_nome,
            i,
            round(resultado["tempo_ms"], 2),
            resultado["qtd_nos_visitados"],
            bool(resultado["comprimento"] > 0),
            resultado["comprimento"],
            round(resultado["memoria_bytes"] / 1024, 2)  # em KB
        ])

    # Salva CSV
    os.makedirs(os.path.dirname(output_csv), exist_ok=True)
    with open(output_csv, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([
            "algoritmo", "tamanho_labirinto", "indice_labirinto",
            "tempo_execucao_ms", "nos_visitados", "encontrou_caminho",
            "comprimento_caminho", "memoria_kb"
        ])
        writer.writerows(resultados)

    print(f"✅ Resultados salvos em: {output_csv}")


## Resolução do lote com labirintos pequenos

In [ ]:
executar_astar_em_lote("labirintos/pequenos", "pequeno", "resultados/astar_manhattan/astar_manhattan_pequenos.csv")


Labirinto 1/20
Labirinto 2/20
Labirinto 3/20
Labirinto 4/20
Labirinto 5/20
Labirinto 6/20
Labirinto 7/20
Labirinto 8/20
Labirinto 9/20
Labirinto 10/20
Labirinto 11/20
Labirinto 12/20
Labirinto 13/20
Labirinto 14/20
Labirinto 15/20
Labirinto 16/20
Labirinto 17/20
Labirinto 18/20
Labirinto 19/20
Labirinto 20/20
✅ Resultados salvos em: resultados/astar_manhattan/astar_manhattan_pequenos.csv


## Resolução do lote com labirintos médios

In [ ]:
executar_astar_em_lote("labirintos/medios", "medio", "resultados/astar_manhattan/astar_manhattan_medios.csv")


Labirinto 1/20
Labirinto 2/20
Labirinto 3/20
Labirinto 4/20
Labirinto 5/20
Labirinto 6/20
Labirinto 7/20
Labirinto 8/20
Labirinto 9/20
Labirinto 10/20
Labirinto 11/20
Labirinto 12/20
Labirinto 13/20
Labirinto 14/20
Labirinto 15/20
Labirinto 16/20
Labirinto 17/20
Labirinto 18/20
Labirinto 19/20
Labirinto 20/20
✅ Resultados salvos em: resultados/astar_manhattan/astar_manhattan_medios.csv


## Resolução do lote com labirintos médios-grandes

In [ ]:
executar_astar_em_lote("labirintos/medio-grande", "medio-grande", "resultados/astar_manhattan/astar_manhattan_medios_grandes.csv")


Labirinto 1/20
Labirinto 2/20
Labirinto 3/20
Labirinto 4/20
Labirinto 5/20
Labirinto 6/20
Labirinto 7/20
Labirinto 8/20
Labirinto 9/20
Labirinto 10/20
Labirinto 11/20
Labirinto 12/20
Labirinto 13/20
Labirinto 14/20
Labirinto 15/20
Labirinto 16/20
Labirinto 17/20
Labirinto 18/20
Labirinto 19/20
Labirinto 20/20
✅ Resultados salvos em: resultados/astar_manhattan/astar_manhattan_medios_grandes.csv


## Resolução do lote com labirintos grandes

In [ ]:
executar_astar_em_lote("labirintos/grandes", "grande", "resultados/astar_manhattan/astar_manhattan_grandes.csv")


Labirinto 1/20
Labirinto 2/20
Labirinto 3/20
Labirinto 4/20
Labirinto 5/20
Labirinto 6/20
Labirinto 7/20
Labirinto 8/20
Labirinto 9/20
Labirinto 10/20
Labirinto 11/20
Labirinto 12/20
Labirinto 13/20
Labirinto 14/20
Labirinto 15/20
Labirinto 16/20
Labirinto 17/20
Labirinto 18/20
Labirinto 19/20
Labirinto 20/20
✅ Resultados salvos em: resultados/astar_manhattan/astar_manhattan_grandes.csv


In [ ]:
from google.colab import files
import shutil

shutil.make_archive("astar_manhatan_results", "zip", "/content/resultados")
files.download("astar_manhatan_results.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>